# Analisando Orgulho e Preconceito com PySpark

## Instalando o PySpark

In [8]:
# !pip install pyspark py4j

In [9]:
# Inicializando uma sessão no PySpark

from pyspark.sql import SparkSession
spark = SparkSession \
          .builder \
          .appName("CDIA4-22-PySpark") \
          .getOrCreate()


spark

## Passo #01. Leitura do livro

In [10]:
book = spark.read.text("books/pride-and-prejudice.txt")
book.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|most other parts ...|
|whatsoever. You m...|
|of the Project Gu...|
|www.gutenberg.org...|
|will have to chec...|
|   using this eBook.|
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Release Date: Jun...|
|[Most recently up...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
+--------------------+
only showing top 20 rows



## Passo #02. Tokenização

In [11]:
from pyspark.sql.functions import split

lines = book.select(
    split(book.value, " ").alias("line")
)
#lines.show()

# Dá pra mudar a truncagem, mostrando mais colunas que o padrão
lines.show(10, truncate=50)

+--------------------------------------------------+
|                                              line|
+--------------------------------------------------+
|[The, Project, Gutenberg, eBook, of, Pride, and...|
|                                                []|
|[This, eBook, is, for, the, use, of, anyone, an...|
|[most, other, parts, of, the, world, at, no, co...|
|[whatsoever., You, may, copy, it,, give, it, aw...|
|[of, the, Project, Gutenberg, License, included...|
|[www.gutenberg.org., If, you, are, not, located...|
|[will, have, to, check, the, laws, of, the, cou...|
|                             [using, this, eBook.]|
|                                                []|
+--------------------------------------------------+
only showing top 10 rows



In [12]:
# Explodindo as linhas (que são compostas por listas)

from pyspark.sql.functions import explode, col

words = lines.select(
    explode(col("line")).alias("word")
)
words.show()

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     eBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows



## Passo #03. Limpeza dos dados

In [13]:
# Mudar a caixa das palavras para minúsculo

from pyspark.sql.functions import lower

lowered_words = words.select(
    lower(
        col("word")
    ).alias("lowered_word")
)
lowered_words.show(15)

+------------+
|lowered_word|
+------------+
|         the|
|     project|
|   gutenberg|
|       ebook|
|          of|
|       pride|
|         and|
|  prejudice,|
|          by|
|        jane|
|      austen|
|            |
|        this|
|       ebook|
|          is|
+------------+
only showing top 15 rows



### Removendo a pontuação

In [14]:
from pyspark.sql.functions import regexp_extract

cleaned_words = lowered_words.select(
    regexp_extract(
        col("lowered_word"), # Indico a coluna
        "[a-z]+", # Seleciona somente o que parece uma palavra
        0 # 0 significa dar o match em toda expressão regular
    ).alias("cleaned_word")
)
cleaned_words.show()

+------------+
|cleaned_word|
+------------+
|         the|
|     project|
|   gutenberg|
|       ebook|
|          of|
|       pride|
|         and|
|   prejudice|
|          by|
|        jane|
|      austen|
|            |
|        this|
|       ebook|
|          is|
|         for|
|         the|
|         use|
|          of|
|      anyone|
+------------+
only showing top 20 rows



### Filtrando os valores NULL

In [15]:
words_nonull = cleaned_words.where(
    col("cleaned_word") != ""
)
words_nonull = words_nonull.select(
    col("cleaned_word").alias("word")
)
words_nonull.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
+---------+
only showing top 20 rows



### Contagem das palavras

In [16]:
counts = words_nonull.groupBy(
    col("word")
).count()
counts.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  208|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   59|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



### Ordenação das contagens

In [17]:
counts.orderBy(
    col("count"),
    ascending=False
).show()

+----+-----+
|word|count|
+----+-----+
| the| 4511|
|  to| 4237|
|  of| 3724|
| and| 3599|
| her| 2224|
|   i| 2052|
|   a| 1997|
|  in| 1922|
| was| 1847|
| she| 1703|
|that| 1583|
|  it| 1542|
| not| 1454|
| you| 1431|
|  he| 1334|
| his| 1268|
|  be| 1256|
|  as| 1193|
| had| 1176|
|with| 1099|
+----+-----+
only showing top 20 rows



# Retomando

## Passo #2.1

In [18]:
words_lst_df = words_nonull.select(
    split(
        col("word"),
        " "
    ).alias("word")
)
words_lst_df.show(10)

+-----------+
|       word|
+-----------+
|      [the]|
|  [project]|
|[gutenberg]|
|    [ebook]|
|       [of]|
|    [pride]|
|      [and]|
|[prejudice]|
|       [by]|
|     [jane]|
+-----------+
only showing top 10 rows



In [19]:
# Remove as stop words
from pyspark.ml.feature import StopWordsRemover

swr = StopWordsRemover(
    inputCol="word",
    outputCol="meaningful"
)

swr_df = swr.transform(words_lst_df)
swr_df.show()

+-----------+-----------+
|       word| meaningful|
+-----------+-----------+
|      [the]|         []|
|  [project]|  [project]|
|[gutenberg]|[gutenberg]|
|    [ebook]|    [ebook]|
|       [of]|         []|
|    [pride]|    [pride]|
|      [and]|         []|
|[prejudice]|[prejudice]|
|       [by]|         []|
|     [jane]|     [jane]|
|   [austen]|   [austen]|
|     [this]|         []|
|    [ebook]|    [ebook]|
|       [is]|         []|
|      [for]|         []|
|      [the]|         []|
|      [use]|      [use]|
|       [of]|         []|
|   [anyone]|   [anyone]|
| [anywhere]| [anywhere]|
+-----------+-----------+
only showing top 20 rows



In [20]:
# Removing the NULLs
meaningful = swr_df.select(
    (col("meaningful")[0]).alias("meaningful")
)
meaningful.show()

+----------+
|meaningful|
+----------+
|      null|
|   project|
| gutenberg|
|     ebook|
|      null|
|     pride|
|      null|
| prejudice|
|      null|
|      jane|
|    austen|
|      null|
|     ebook|
|      null|
|      null|
|      null|
|       use|
|      null|
|    anyone|
|  anywhere|
+----------+
only showing top 20 rows



In [21]:
# Filtrar os valores NULLs
meaningful_nonull = meaningful.where(
    col("meaningful") != "null"
)
meaningful_nonull.show(5)

+----------+
|meaningful|
+----------+
|   project|
| gutenberg|
|     ebook|
|     pride|
| prejudice|
+----------+
only showing top 5 rows



### Recontagem dos valores

In [22]:
words_counts = meaningful_nonull.groupby(
    col("meaningful")
).count()

words_counts.show()

+-------------+-----+
|   meaningful|count|
+-------------+-----+
|       online|    4|
|        still|   72|
|         hope|  122|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
|   likelihood|    5|
|        pools|    1|
|       coaxed|    1|
+-------------+-----+
only showing top 20 rows



In [23]:
# Classificação da contagem
words_counts = words_counts.orderBy(
    col("count"), 
    ascending=False
)
words_counts.show()


+----------+-----+
|meaningful|count|
+----------+-----+
|        mr|  783|
| elizabeth|  633|
|     darcy|  418|
|      said|  402|
|       mrs|  344|
|      much|  329|
|    bennet|  324|
|      must|  318|
|   bingley|  307|
|      jane|  294|
|      miss|  283|
|       one|  276|
|      know|  239|
|      well|  224|
|    though|  221|
|     never|  220|
|    sister|  218|
|      soon|  215|
|     think|  211|
|       may|  207|
+----------+-----+
only showing top 20 rows

